In [1]:
!pip install PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import category_encoders as ce

RANDOM_SEED = 42


df = pd.read_csv("data/adult.csv")

# Transform the target variable to 0,1

mapping = {"<=50K": 0, ">50K": 1}
df["income"]  = df["income"].map(mapping)

X = df.drop("income",axis=1)
y = df["income"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)



In [14]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2

def show_column_position(dat):
    font = ImageFont.truetype("Arial.ttf", size=20)
    background = np.array([[0 for _ in range(255)] for _ in range(255)], dtype='uint8')
    image = Image.fromarray(background)
    draw = ImageDraw.Draw(image)
    draw.text((32, 32), str(dat[0][:12]), fill='white', font=font)
    draw.text((32, 160), str(dat[1][:11]), fill='white', font=font)
    draw.text((160, 32), str(dat[2][:11]), fill='white', font=font)
    draw.text((160, 160), str(dat[3][:11]), fill='white', font=font)
    rgb = [np.array(image, dtype='uint8') for _ in range(3)]
    return np.array(rgb)

def show_column_position_another(dat):
    font = ImageFont.truetype("Arial.ttf", size=20)
    background = np.array([[0 for _ in range(255)] for _ in range(255)], dtype='uint8')
    image = Image.fromarray(background)
    draw = ImageDraw.Draw(image)
    draw.text((32, 32), str(dat[2][:12]), fill='white', font=font)
    draw.text((32, 160), str(dat[3][:11]), fill='white', font=font)
    draw.text((160, 32), str(dat[1][:11]), fill='white', font=font)
    draw.text((160, 160), str(dat[0][:11]), fill='white', font=font)
    rgb = [np.array(image, dtype='uint8') for _ in range(3)]
    return np.array(rgb)


def data_to_image_another(data):
    data_images = []
    font = ImageFont.truetype("Arial.ttf", size=50)
    for dat in data:
        background = np.array([[0 for _ in range(255)] for _ in range(255)], dtype='uint8')
        image = Image.fromarray(background)
        draw = ImageDraw.Draw(image)
        draw.text((32, 32), str(dat[2]), fill='white', font=font)
        draw.text((32, 160), str(dat[3]), fill='white', font=font)
        draw.text((160, 32), str(dat[1]), fill='white', font=font)
        draw.text((160, 160), str(dat[0]), fill='white', font=font)
        rgb = [np.array(image, dtype='uint8') for _ in range(3)]
        data_images.append(rgb)
    return np.array(data_images) / 255


In [6]:
import torch.nn as nn
from torchvision import models
import matplotlib.pyplot as plt

model_res = models.resnet18(weights=True)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 3)

In [61]:
def data_to_image(data):
    
    data_images = []
    font = ImageFont.truetype("arial.ttf", size=50)
    for dat in data.values:
        background = np.array([[0 for _ in range(255)] for _ in range(255)], dtype='uint8')
        image = Image.fromarray(background)
        draw = ImageDraw.Draw(image)
        
        y_offset = 32  # Décalage initial en y
        x_offset = 32  # Décalage initial en x
        
        for i, value in enumerate(dat):
            draw.text((x_offset, y_offset), str(value), fill='white', font=font)
            y_offset += 128  # Ajustement de la position en y pour la prochaine valeur
            if i % 2 != 0:
                y_offset = 32  # Réinitialisation du décalage en y après chaque deuxième valeur
                x_offset += 128  # Décalage en x pour la prochaine ligne

        rgb = [np.array(image, dtype='uint8') for _ in range(3)]
        data_images.append(rgb)
    return np.array(data_images) / 255

In [ ]:
""" from sklearn.datasets import load_iris
import torch
from torch.utils.data import DataLoader, TensorDataset

iris_data = load_iris()

X_train, X_val, y_train, y_val = train_test_split(iris_data.data, iris_data.target, stratify=iris_data.target, test_size=0.2, random_state=0)

train_images = data_to_image(X_train)
val_images = data_to_image(X_val)

""" print(train_images.shape)
print(val_images.shape)
plt.grid()
plt.imshow(train_images[2][0, :, :]) """


X_train = torch.from_numpy(train_images).float()
y_train = torch.from_numpy(y_train).long()
X_val = torch.from_numpy(val_images).float()
y_val = torch.from_numpy(y_val).long()

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

dataloaders = {'train': DataLoader(train_dataset, batch_size=16, shuffle=True),
              'val': DataLoader(val_dataset)}

dataset_sizes = {'train': len(X_train),
                'val': len(X_val)} """

In [ ]:
X_train.values

In [62]:
train_images = data_to_image(X_train)
test_images = data_to_image(X_test)

print(train_images.shape)


plt.grid()
plt.imshow(train_images[1][2, :, :])

MemoryError: Unable to allocate 7.10 GiB for an array with shape (39073, 3, 255, 255) and data type uint8

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

X_train = torch.from_numpy(train_images).float()
y_train = torch.from_numpy(y_train.squeeze().to_numpy()).long()
X_test = torch.from_numpy(test_images).float()
y_test = torch.from_numpy(y_test.to_numpy()).long()

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_test, y_test)

dataloaders = {'train': DataLoader(train_dataset, batch_size=16, shuffle=True),
              'val': DataLoader(val_dataset)}

dataset_sizes = {'train': len(X_train),
                'val': len(X_test)}

AssertionError: Size mismatch between tensors

#### Echec de la mission problème de mémoire pour de grands datasets